In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import CosineRecommender
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix

In [2]:
data_dir = "/Users/ur001/Documents/Datasets/lastfm-dataset-360K"
dest_data_dir = "/Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest"

In [3]:
def load_data(path): 
    return pd.read_table(
        os.path.join(path, "usersha1-artmbid-artname-plays.tsv"),
        usecols=[0, 2, 3],
        names=['user', 'artist', 'plays']
    )

In [4]:
def prepare_data(data):
    """Возвращает подготовленные данные и словарь артистов"""
    data.dropna(inplace=True)  
    data['user_id'] = data.user.astype("category").cat.codes.copy() + 1
    data['artist_id'] = data.artist.astype("category").cat.codes.copy() + 1
    artists = dict(enumerate(data.artist.astype("category").cat.categories))
    return data[['user_id', 'artist_id', 'plays']], artists

In [5]:
from sklearn.model_selection import train_test_split

def split_data(data, size=0.2):
    data_train, data_test = train_test_split(data, test_size=size)
    test_user_set = set(data_test.user_id.unique())
    train_user_set = set(data_train.user_id.unique())
    # Оставляем только пользователей которые есть одновременно в тестовой и обучающей выборке
    user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
    return (
        data_train[~data_train.user_id.isin(user_ids_to_exclude)], 
        data_test[~data_test.user_id.isin(user_ids_to_exclude)]
    )

In [6]:
def save_data(path, name, data, columns=["user_id", "artist_id", "plays"]):
    file_name = os.path.join(path, name + ".tsv")
    data[columns].to_csv(file_name, sep="\t", header=False, index=False)

In [7]:
def make_coo_matrix(data):
    """Создаёт разреженную матрицу item*user"""
    return coo_matrix((
        data.plays.astype(np.double),
        (data.artist_id, data.user_id)
    ))

def sparse_info(sparse_matrix):
    """функция, которая красиво печатает информацию о разреженных матрицах"""
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}".format(
        sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1]
    ))
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [8]:
def make_user_recommendations(model, data_train_coo, data_test, path, model_name):
    """Генерирует рекомендации для каждого пользователя и сохраняет их в файл для проверки mrec-ом"""
    user_plays = data_train_coo.T.tocsr()
    file_name = "test.{}.tsv.recs.tsv".format(model_name)
    # Получаем столько рекомендаций, сколько записей у пользователя в тестовой выборке, но не меньше 3
    user_rec_counts = dict(data_test.user_id.value_counts())
    user_rec_counts = {user_id: max(10, int(N)) for user_id, N in user_rec_counts.items()}
    with open(os.path.join(path, file_name), "w") as output_file:
        for user_id in tqdm_notebook(data_test.user_id.unique()):
            for artist_id, score in model.recommend(user_id, user_plays, N=user_rec_counts[user_id]):
                output_file.write("{}\t{}\t{}\n".format(user_id, artist_id, score))    

In [9]:
def search_artist(artists, name_prefix):
    """Поиск по базе исполнителей: выводит список всех начинающихся с переданного префикса"""
    for artist_id, artist_name in artists.items():
        if artist_name.lower().startswith(name_prefix):
            print("[{}]\t{}".format(artist_id, artist_name))
            
def find_similar_artists(model, artist_id, artists, count=10):
    """Выводит список похожих исполнителей"""
    print("Similar to {}:".format(artists[artist_id]))
    print("-------------------------------")
    for similar_artist_id, score in model.similar_items(artist_id + 1, count):
        print("{:.3f}\t{}".format(score, artists[similar_artist_id - 1]))

In [10]:
data = load_data(data_dir)

In [11]:
data, artists = prepare_data(data)

In [12]:
data.head()

,user_id,artist_id,plays
0,1,45562,2137
1,1,90934,1099
2,1,185368,897
3,1,106705,717
4,1,155242,706


In [13]:
data_train, data_test = split_data(data, 0.2)

In [14]:
data_train.head()

,user_id,artist_id,plays
6038389,123552,91865,70
1868467,38216,37976,44
1054556,21574,119476,433
7841147,160403,204338,130
11937809,244288,54081,6


In [15]:
save_data(dest_data_dir, 'train', data_train)

In [16]:
data_train_coo = make_coo_matrix(data_train)

In [17]:
sparse_info(data_train_coo)

Размерности матрицы: (292365, 358869)
Ненулевых элементов в матрице: 14025403
Доля ненулевых элементов: 0.0001336761792153849
Среднее значение ненулевых элементов: 215.21585340542444
Максимальное значение ненулевых элементов: 272359.0
Минимальное значение ненулевых элементов: 0.0


In [18]:
# строим матрицу схожести по косинусной мере
models = {}
models['knn_cosine'] = CosineRecommender()
models['knn_cosine'].fit(data_train_coo)

/Users/ur001/.pyenv/versions/anaconda3-5.0.0/envs/netology1/lib/python3.6/site-packages/implicit/nearest_neighbours.py:109: RuntimeWarning: invalid value encountered in true_divide
  X.data = X.data / sqrt(bincount(X.row, X.data ** 2))[X.row]


In [20]:
save_data(dest_data_dir, 'test.knn_cosine', data_test)

In [21]:
# Попробуем найти похожих исполнителей на Aphex Twin, чтобы убедиться что рекомендации вообще работают
search_artist(artists, 'aphex twin')

[29190]	aphex twin
[29191]	aphex twin & céline eia
[29192]	aphex twin & m-ziq (aka rich & mike)
[29193]	aphex twin & squarepusher
[29194]	aphex twin & µ-ziq (mike & rich)
[29195]	aphex twin & µ-ziq - mike and rich
[29196]	aphex twin (aka afx)
[29197]	aphex twin (aka caustic window)
[29198]	aphex twin (aka polygon window)
[29199]	aphex twin (aka universal indicator)
[29200]	aphex twin - power-pill
[29201]	aphex twin ft. photek & autechre
[29202]	aphex twin vs luke vibert
[29203]	aphex twin vs run jeremy
[29204]	aphex twin/afx
[29205]	aphex twin/luke vibert
[29206]	aphex twin/richard james


In [22]:
find_similar_artists(models['knn_cosine'], 29190, artists)

Similar to aphex twin:
-------------------------------
1.000	aphex twin
0.289	squarepusher
0.239	afx
0.232	boards of canada
0.207	autechre
0.193	venetian snares
0.190	bobby beausoleil & the freedom orchestra
0.188	chris huelsbeck cd4
0.173	antonius rex
0.169	plaid


In [23]:
# Создаём рекомендации для всех тестовых пользователей
make_user_recommendations(models['knn_cosine'], data_train_coo, data_test, dest_data_dir, 'knn_cosine')

HBox(children=(IntProgress(value=0, max=358511), HTML(value='')))

In [24]:
# Проверяем рекомендации mrec-ом
!/Users/ur001/.pyenv/versions/netology1/bin/mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest/test.knn_cosine.tsv \
    --recsdir /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest

[2017-12-04 15:03:26,417] INFO: processing /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest/test.knn_cosine.tsv...
None
mrr            0.1142 +/- 0.0000
prec@5         0.0411 +/- 0.0000
prec@10        0.0333 +/- 0.0000
prec@15        0.0243 +/- 0.0000
prec@20        0.0184 +/- 0.0000


In [32]:
# Тренируем ALS-модель
models['als'] = AlternatingLeastSquares(factors=64)
models['als'].fit(data_train_coo)

In [25]:
# Проверяем модель на Aphex Twin-е
find_similar_artists(models['als'], 29190, artists)

Similar to aphex twin:
-------------------------------
1.000	aphex twin
0.821	boards of canada
0.793	squarepusher
0.762	amon tobin
0.736	autechre
0.686	four tet
0.671	portishead
0.671	air
0.659	dj shadow
0.650	björk


In [30]:
make_user_recommendations(models['als'], data_train_coo, data_test, dest_data_dir, 'als')

HBox(children=(IntProgress(value=0, max=358540), HTML(value='')))

In [ ]:
save_data(dest_data_dir, 'test.als', data_test)

In [31]:
!/Users/ur001/.pyenv/versions/netology1/bin/mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest/test.als.tsv \
    --recsdir /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest

[2017-12-03 22:46:55,419] INFO: processing /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest/test.als.tsv...
None
mrr            0.0804 +/- 0.0000
prec@5         0.0291 +/- 0.0000
prec@10        0.0243 +/- 0.0000
prec@15        0.0162 +/- 0.0000
prec@20        0.0122 +/- 0.0000
